In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json
from flatten_json import flatten

In [3]:
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

apikey = "RGAPI-502fbac9-234f-47fc-b37e-23b74ef563b0"

In [ ]:
# get challenger summonerId from league-exp-api
# 1.2 delay bc short list

req = requests.get("https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=1&api_key={}"
                   .format(apikey),time.sleep(1.2))
assert req.status_code == 200
data = req.json()
sumids = [d['summonerId'] for d in data]

In [ ]:
sumids

In [ ]:
# get accountId via summonerId via summoner-api
# 1.2 delay bc short list

accids = []
for sumid in sumids:
    sumreq = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}"
                          .format(sumid,apikey),time.sleep(1.2))
    assert sumreq.status_code == 200
    accids.append(sumreq.json()['accountId'])

In [ ]:
accids

In [ ]:
# get game ids from match history via match-api
# 1.3 delay bc long list

gamids = []

for acc in accids:
    matlreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.3))
    
    i=0
    while matlreq.status_code != 200:
        matlreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            break
    
    gamids += [d['gameId'] for d in matlreq.json()['matches']]

In [ ]:
matlreq.status_code

In [ ]:
len(gamids)

In [ ]:
# print original length, then correct overlap from shared games and print new length

print(len(gamids))

gamids = set(gamids)

print(len(gamids))

In [ ]:
with open('euw_chall_gam_id_list.txt', 'r') as f:
    challgames = f.read().splitlines()

In [ ]:
challgames =set([int(e) for e in challgames])

In [ ]:
len(challgames)

In [ ]:
len(gamids)

In [ ]:
len(gamids - (challgames & gamids))

In [ ]:
gamids = gamids - (challgames & gamids)

In [ ]:
with open('gm_gam_id_list.txt', 'w') as f:
    for item in gamids:
        f.write("%s\n" % item)

In [8]:
with open('gm_gam_id_list.txt', 'r') as f:
    ids = f.read().splitlines()
    
gamids = list(set([int(e) for e in ids]))

In [22]:
len(gamids)

9914

In [34]:
# get individual match information from gamids list via match-api
# 1.5 delay bc long list and repeated issues with 

matchlist = []

for gam in gamids[9332:9914]:
    gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
    
    i=0
    while gamreq.status_code != 200:
        gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            break
            
    match = flatten(gamreq.json())
    matchlist.append(match)

In [35]:
gamreq.status_code

200

In [36]:
# check length of matchlist 

check = [len(m) for m in matchlist]
len(matchlist),max(check),min(check)

(582, 1550, 1210)

In [37]:
# create df from matches

# df = pd.DataFrame(matchlist)

In [38]:
df1 = pd.DataFrame(matchlist)

In [40]:
df = df.append(df1)

In [41]:
df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participants_9_timeline_csDiffPerMinDeltas_30-end,participants_9_timeline_csDiffPerMinDeltas_20-30,participants_9_timeline_xpDiffPerMinDeltas_30-end,participants_9_timeline_xpDiffPerMinDeltas_20-30,participants_9_timeline_damageTakenPerMinDeltas_30-end,participants_9_timeline_damageTakenPerMinDeltas_20-30,participants_9_timeline_damageTakenDiffPerMinDeltas_30-end,participants_9_timeline_damageTakenDiffPerMinDeltas_20-30,teams_0_bans,teams_1_bans
0,4660841737,EUW1,1592171880704,845,900,11,13,10.12.324.5925,URF,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4669623562,EUW1,1592653463718,1560,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4669459725,EUW1,1592651307330,2289,440,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,0.4,3.3,-522.0,255.50,1891.4,1647.1,418.6,803.5,NaN,NaN
3,4673785106,EUW1,1592859584623,954,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4601302292,EUW1,1589543736086,1809,420,11,13,10.10.320.3039,CLASSIC,MATCHED_GAME,...,NaN,-0.5,NaN,105.35,NaN,461.4,NaN,-181.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,4723605480,EUW1,1595594040069,838,1300,21,13,10.15.328.8759,NEXUSBLITZ,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
578,4696244211,EUW1,1594138740718,838,450,12,13,10.13.326.4870,ARAM,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
579,4700602358,EUW1,1594359721851,1903,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,2.8,NaN,-299.40,NaN,1174.5,NaN,145.9,NaN,NaN
580,4709351417,EUW1,1594841284913,1147,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
gmgames = list(df['gameId'])

In [ ]:
len(set(list(df['gameId'])))

In [42]:
df.to_csv('eugm9000.zip', index=False,
          compression = dict(method='zip',archive_name='eugm3.csv') )